In [13]:
#uncomment and run to install spacy
#import sys
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m spacy download en

In [1]:
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('en_core_web_sm')

[List of all named entities](https://spacy.io/api/annotation#named-entities)

### Extract `MONEY` and `PRODUCT` from the earning calls

### Extract all named entities from the earning calls. Rank them by word frequencies

In [3]:
data = pd.read_csv('data/EC10.csv')

In [4]:
data.head()

,id,text,date,company,sector
0,32934,"Zoe's Kitchen, Inc. (NYSE:ZOES)\r\nQ2 2015 Ear...",2015-08-28 00:11:00,"Zoe's Kitchen, Inc.",Consumer Services
1,32905,"Autodesk, Inc. (NASDAQ:ADSK)\r\nQ2 2016 Earnin...",2015-08-28 00:07:00,"Autodesk, Inc.",Technology
2,32926,Smith & Wesson Holding Corp. (NASDAQ:SWHC)\r\n...,2015-08-27 23:25:00,Smith & Wesson Holding Corporation,Capital Goods
3,32930,"Ulta Salon, Cosmetics & Fragrance, Inc. (NASDA...",2015-08-27 23:09:00,"Ulta Salon, Cosmetics & Fragrance, Inc.",Consumer Services
4,32907,Splunk Inc. (NASDAQ:SPLK)\r\nQ2 2016 Earnings ...,2015-08-27 22:56:00,Splunk Inc.,Technology


In [5]:
text = data.iloc[0, 1]

In [31]:
def summarize(text, selected_entities = {"MONEY", "PRODUCT"}, top_n = 10):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    filtered = [text for text, label in entities if label in selected_entities]
    return pd.Series(filtered).value_counts()[:top_n].to_dict()

In [32]:
data['summary'] = data['text'].apply(summarize)

In [34]:
data['text'].apply(lambda text:
                   summarize(text,selected_entities = {"PERSON"}) )

0    {'Kevin Miles - President': 18, 'Kevin Miles':...
1    {'Carl Bass - President': 48, 'R. Scott Herren...
2    {'P. James Debney - President': 30, 'Jeffrey D...
3    {'Mary N. Dillon - Chief': 14, 'Scott M. Sette...
4    {'Godfrey Sullivan': 22, 'Splunk Cloud': 12, '...
5    {'Julian Paul Raines -': 57, 'Tony D. Bartel':...
6    {'Aurelio Cisneros': 10, 'Anson Chan': 7, 'Tho...
7    {'Colleen Johnston': 13, 'TD': 11, 'Mark Chauv...
8    {'Todd Noden': 4, 'Terry Finley': 3, 'Harper L...
9    {'Peter Gassner': 15, 'Brendan Barnicle': 6, '...
Name: text, dtype: object

In [35]:
_.to_list()

[{'Kevin Miles - President': 18,
  'Kevin Miles': 16,
  'James Besch - CFO': 14,
  'Kevin': 10,
  'Paul Westra - Stifel': 8,
  'Jim': 6,
  'Andrew Marc': 5,
  'Karen F. Short - Deutsche Bank Securities, Inc.': 4,
  'Nicole M. Miller Regan - Piper Jaffray & Co': 4,
  'Karen F. Short - Deutsche Bank': 3},
 {'Carl Bass - President': 48,
  'R. Scott Herren': 22,
  'Carl': 13,
  'R. Scott Herren -': 11,
  'Steve R. Koenig -': 11,
  'Scott': 6,
  'Autodesk': 6,
  'Anne Bellini': 6,
  'Jay Vleeschhouwer - Griffin Securities,': 5,
  'Jay Vleeschhouwer - Griffin Securities, Inc.': 5},
 {'P. James Debney - President': 30,
  'Jeffrey D. Buchanan - Executive': 15,
  'Financial Officer': 15,
  'Scott L. Stember - C.L. King & Associates': 10,
  'Jeff': 9,
  'Cai von Rumohr - Cowen & Co.': 9,
  'James': 7,
  'Steven L. Cahall -': 6,
  'Brian W. Ruttenbur -': 4,
  'Brian': 4},
 {'Mary N. Dillon - Chief': 14,
  'Scott M. Settersten -': 11,
  'Mary': 9,
  'David C. Kimbell': 8,
  'Lancôme': 5,
  'Scott'

['GAAP', 'GM Conference']